In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

In [3]:
# Create the first dataframe
df1 = spark.createDataFrame([
    ("A", 1), 
    ("B", 2), 
    ("C", 3)], ["letter", "number"])

# Create the second dataframe
df2 = spark.createDataFrame([
    ("A", 4), 
    ("B", 5), 
    ("D", 6)], ["letter", "value"])

## 1. Inner join

In [4]:
# Perform the inner join
inner_join = df1.join(df2, df1['letter'] == df2['letter'], "inner")

# Show the result of the join
inner_join.show()

+------+------+------+-----+
|letter|number|letter|value|
+------+------+------+-----+
|     A|     1|     A|    4|
|     B|     2|     B|    5|
+------+------+------+-----+



## 2. Left outer join

In [5]:

left_outer_join = df1.join(df2, df1['letter'] == df2['letter'], "left_outer")
# Show the result of the join
left_outer_join.show()

+------+------+------+-----+
|letter|number|letter|value|
+------+------+------+-----+
|     A|     1|     A|    4|
|     B|     2|     B|    5|
|     C|     3|  NULL| NULL|
+------+------+------+-----+



## 3. Right outer join

In [6]:
# Perform the right outer join
right_outer_join = df1.join(df2, df1['letter'] == df2['letter'], "right_outer")
# Show the result of the join
right_outer_join.show()

+------+------+------+-----+
|letter|number|letter|value|
+------+------+------+-----+
|     A|     1|     A|    4|
|     B|     2|     B|    5|
|  NULL|  NULL|     D|    6|
+------+------+------+-----+



## 4. Full outer join

In [7]:
# Perform the full outer join
full_outer_join = df1.join(df2, df1['letter'] == df2['letter'], "full_outer")
# Show the result of the join
full_outer_join.show()

+------+------+------+-----+
|letter|number|letter|value|
+------+------+------+-----+
|     A|     1|     A|    4|
|     B|     2|     B|    5|
|     C|     3|  NULL| NULL|
|  NULL|  NULL|     D|    6|
+------+------+------+-----+



## 5. Cross join

In [8]:
# Perform the cross join
cross_join = df1.crossJoin(df2)
# Show the result of the join
cross_join.show()

+------+------+------+-----+
|letter|number|letter|value|
+------+------+------+-----+
|     A|     1|     A|    4|
|     A|     1|     B|    5|
|     A|     1|     D|    6|
|     B|     2|     A|    4|
|     B|     2|     B|    5|
|     B|     2|     D|    6|
|     C|     3|     A|    4|
|     C|     3|     B|    5|
|     C|     3|     D|    6|
+------+------+------+-----+



## 6. Left Anti Join

In [9]:
# Perform the left anti join
left_anti_join = df1.join(df2, df1['letter'] == df2['letter'], "left_anti")
# Show the result of the join
left_anti_join.show()

+------+------+
|letter|number|
+------+------+
|     C|     3|
+------+------+



## 7. Left Semi Join

In [10]:
# Perform the left semi join
left_semi_join = df1.join(df2, df1['letter'] == df2['letter'], "leftsemi")
# Show the result of the join
left_semi_join.show()

+------+------+
|letter|number|
+------+------+
|     A|     1|
|     B|     2|
+------+------+



## 8. Self Join

In [11]:
# Perform the self join
self_join = df1.join(df1.alias("df2"), df1["letter"] == df1["letter"])
# Show the result of the join
self_join.show()

+------+------+------+------+
|letter|number|letter|number|
+------+------+------+------+
|     A|     1|     A|     1|
|     B|     2|     B|     2|
|     C|     3|     C|     3|
+------+------+------+------+



In [12]:
impressions = (
  spark
    .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
    .selectExpr("value AS adId", "timestamp AS impressionTime")
)

In [13]:
display(impressions)

DataFrame[adId: bigint, impressionTime: timestamp]